In [1]:
import os
from GetDataLoaders import get_dataloaders, get_short_dataloaders
from architectures.AlexNetFeature import AlexNetFeature
from architectures.AlexNetClassifier import AlexNetClassifier
from architectures.LinearTransformationNorm import LinearTransformationNorm
from architectures.NCEAverage import NCEAverage
from architectures.NCELoss import NCELoss
import torch
import numpy as np
#from torch.utils.tensorboard import SummaryWriter
from torch import nn
import time
from torch import optim

In [2]:
'''
# we skip the probs for now
gama = 2.0
with open(os.path.join("./PUprobs", 'prob.dat'), 'r') as file_input:
    train_prob_str = file_input.readlines()
    train_prob = [float(i_prob_str.rstrip('\n')) for i_prob_str in train_prob_str]
    print(len(train_prob)/4.0)
    train_weight = [1.0 if 0==i%4 else 1-train_prob[i]**gama for i in range(len(train_prob))]
'''

'\n# we skip the probs for now\ngama = 2.0\nwith open(os.path.join("./PUprobs", \'prob.dat\'), \'r\') as file_input:\n    train_prob_str = file_input.readlines()\n    train_prob = [float(i_prob_str.rstrip(\'\n\')) for i_prob_str in train_prob_str]\n    print(len(train_prob)/4.0)\n    train_weight = [1.0 if 0==i%4 else 1-train_prob[i]**gama for i in range(len(train_prob))]\n'

In [3]:
use_cuda = torch.cuda.is_available()
device = "cuda" if use_cuda else "cpu"
batch_size = 192
lr = 0.1
LUT_lr = [(90,0.01), (130,0.001), (190,0.0001), (210,0.00001), (230,0.0001), (245,0.00001)]
num_epochs = 245
momentum = 0.9
weight_decay = 5e-4
nesterov = True
Lambdas = {'CE':1.0, 'MSE':1.0, 'NCE':1.0}

loaders = get_dataloaders('imagenet', batch_size=batch_size, num_workers=2)
ndata_train = len(loaders['train_loader'].dataset)
ndata_valid = len(loaders['valid_loader'].dataset)
t = 0.07
m = 4096
gamma = 2

In [4]:
ndata_train, ndata_valid

(80000, 10000)

In [5]:
feature_net = AlexNetFeature().to(device)
classifier_net = AlexNetClassifier().to(device)
transformation_net = LinearTransformationNorm().to(device)

feature_optimizer = optim.SGD(feature_net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
classifier_optimizer = optim.SGD(classifier_net.parameters() ,lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
transformation_optimizer = optim.SGD(transformation_net.parameters() ,lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)

Networks = {'feature':feature_net, 'classifier':classifier_net, 'transformation':transformation_net}
Optimizers = {'feature':feature_optimizer, 'classifier':classifier_optimizer, 'transformation':transformation_optimizer}

NCE = {'NCEAverage_train': NCEAverage(outputSize=ndata_train, K=m, T=t).cuda(), 'NCEAverage_valid': NCEAverage(outputSize=ndata_valid, K=m, T=t).cuda() , 'NCECriterion_train': NCELoss(ndata_train), 'NCECriterion_valid': NCELoss(ndata_valid)}
Criterions = {'CE': nn.CrossEntropyLoss(reduction='none'), 'MSE':nn.MSELoss() }

Moving NCE To CUDA.
Moving AliasMethod To CUDA.
Moving NCE To CUDA.
Moving AliasMethod To CUDA.


In [6]:
#for sample in loaders['train_loader']:
    #print(sample[0].size(), sample[1].size(), sample[2].size())

In [6]:
def adjust_learning_rates(epoch):
    # filter out the networks that are not trainable and that do
    # not have a learning rate Look Up Table (LUT_lr) in their optim_params
    lr = next((lr for (max_epoch, lr) in LUT_lr if max_epoch>epoch), LUT_lr[-1][1])
    for key in Optimizers:
        for g in Optimizers[key].param_groups:
            g['lr'] = lr

def AlexNetDecoupling(batch, NCEAverage, NCECriterion, PUWeights=None, train=True):
    data, targets, indices = batch
    
    if train is True:
        Optimizers['feature'].zero_grad()
        Optimizers['classifier'].zero_grad()
        Optimizers['transformation'].zero_grad()
    
    #to cuda
    
    data = data.to(device)
    targets = targets.to(device)
    indices = indices.to(device)

    
    #perform rotations & adjust data shape
    data_90 = torch.flip(torch.transpose(data,2,3),[2])
    data_180 = torch.flip(torch.flip(data,[2]),[3])
    data_270 = torch.transpose(torch.flip(data,[2]),2,3)
    data = torch.stack([data, data_90, data_180, data_270], dim=1)
    batch_size, rotations, channels, height, width = data.size()
    data = data.view([batch_size*rotations, channels, height, width])
    
    #debug for backward
    data.requires_grad = False 
    targets.requires_grad = False
    indices.requires_grad = False
    
    #collect features
    features = Networks['feature'](data)
    features_rot, features_invariance = torch.split(features, 2048, dim=1)
    
    #collect rotation prediction
    pred = Networks['classifier'](features_rot)
    
    
    #average features across 4 rotations
    features_invariance_instance = features_invariance[0::4,:] + features_invariance[1::4,:] + features_invariance[2::4,:] + features_invariance[3::4,:]
    features_invariance_instance = torch.mul(features_invariance_instance, 0.25) #fbar 192x2048
    
    #downsample to 128 & perform normalization of vector
    features_128_norm = Networks['transformation'](features_invariance_instance) #192x128
    
    with torch.no_grad():
        #stack 192x2048 4 times to be  = 192x4x2048 = 768x2048
        features_invariance_instance_mean = torch.unsqueeze(features_invariance_instance,1).expand(-1,4,-1).clone()
        features_invariance_instance_mean = features_invariance_instance_mean.view(4*len(features_invariance_instance),2048)
    
    #calculate rotation loss ignore PU for now
    loss_cls_each = Criterions['CE'](pred, targets)
    loss_cls = torch.sum(loss_cls_each)/loss_cls_each.shape[0]
    
    #calculate rotation invariance by MSE
    loss_mse = Criterions['MSE'](features_invariance, features_invariance_instance_mean)
    
    #calculate instance loss using NCE
    output_nce = NCEAverage(features_128_norm, indices)
    loss_nce = NCECriterion(output_nce, indices)
    
    #loss_nce = torch.FloatTensor([0.0]).cuda() #NCECriterion(features_128_norm, indices)
    
    loss_total = Lambdas['CE']*loss_cls + Lambdas['MSE']*loss_mse + Lambdas['NCE']*loss_nce
    
    if train is True:
        loss_total.backward()
        Optimizers['feature'].step()
        Optimizers['classifier'].step()
        Optimizers['transformation'].step()
    
    #calculate classification accuracy
    pred = pred.argmax(dim=1, keepdim=True)
    correct = pred.eq(targets.view_as(pred)).sum().item()
    
    losses = {'ce':loss_cls.item(), 'mse':loss_mse.item(), 'nce':loss_nce.item(), 'correct':correct}
    
    return losses
    

In [7]:
def train(data_loader, epoch, log_interval=50):
    
    Networks['feature'].train()
    Networks['classifier'].train()
    Networks['transformation'].train()
    
    losses = {'ce':[], 'mse':[], 'nce':[]}
    correct = 0
    train_loss = np.Inf
    train_acc = 0.0
    adjust_learning_rates(epoch)
    start_time = time.time()
    for batch_idx, sample in enumerate(data_loader):
        
        lossesdict = AlexNetDecoupling(sample, NCE['NCEAverage_train'], NCE['NCECriterion_train'] ,train=True)
        
        losses['ce'].append(lossesdict['ce'])
        losses['mse'].append(lossesdict['mse'])
        losses['nce'].append(lossesdict['nce'])
        correct += lossesdict['correct']
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: CE {:.6f}, MSE {:.6f}, NCE {:.6f}'.format(epoch, batch_idx*len(sample[0]), len(data_loader.dataset), 100. * batch_idx / len(data_loader), lossesdict['ce'], lossesdict['mse'], lossesdict['nce']))
    
    end_time = time.time()
    print("Time for epoch pass {}".format(end_time-start_time))
    train_loss = {'ce': float(np.mean(losses['ce'])), 'mse': float(np.mean(losses['mse'])), 'nce':float(np.mean(losses['nce']))}
    train_acc = correct / float(len(data_loader.dataset)*4)
    print('Train set: Average loss: CE {:.4f}, MSE {:.4f}, NCE {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(train_loss['ce'], train_loss['mse'], train_loss['nce'], correct, len(data_loader.dataset)*4, 100*train_acc))
    return train_loss, train_acc

def validate(data_loader, epoch, log_interval=50):
    
    Networks['feature'].eval()
    Networks['classifier'].eval()
    Networks['transformation'].eval()
    
    losses = {'ce':[], 'mse':[], 'nce':[]}
    correct = 0
    valid_loss = np.Inf
    valid_acc = 0.0
    start_time = time.time()
    for batch_idx, sample in enumerate(data_loader): 
        with torch.no_grad():
            lossesdict = AlexNetDecoupling(sample, NCE['NCEAverage_valid'], NCE['NCECriterion_valid'], train=False)
        
        losses['ce'].append(lossesdict['ce'])
        losses['mse'].append(lossesdict['mse'])
        losses['nce'].append(lossesdict['nce'])
        correct += lossesdict['correct']
        if batch_idx % log_interval == 0:
            print('Valid Epoch: {} [{}/{} ({:.0f}%)]\tLoss: CE {:.6f}, MSE {:.6f}, NCE {:.6f}'.format(epoch, batch_idx*len(sample[0]), len(data_loader.dataset), 100. * batch_idx / len(data_loader), lossesdict['ce'], lossesdict['mse'], lossesdict['nce']))
    
    end_time = time.time()
    print("Time for epoch pass {}".format(end_time-start_time))
    valid_loss = {'ce': float(np.mean(losses['ce'])), 'mse': float(np.mean(losses['mse'])), 'nce':float(np.mean(losses['nce']))}
    valid_acc = correct / float(len(data_loader.dataset)*4)
    print('Valid set: Average loss: CE {:.4f}, MSE {:.4f}, NCE {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(valid_loss['ce'], valid_loss['mse'], valid_loss['nce'], correct, len(data_loader.dataset)*4, 100*valid_acc))
    return valid_loss, valid_acc

def run_main_loop(loaders, num_epochs):
    #writer = SummaryWriter('./logs/AlexNet_Unsupervised_Decoupling')
    save_path = "weights/AlexNet_Decoupling.pth"
    best_acc = 0.0
    for epoch in range(num_epochs):
        print("Performing {}th epoch".format(epoch))
        train_loss, train_acc = train(loaders['train_loader'], epoch)
        val_loss, val_acc = validate(loaders['valid_loader'], epoch)
        '''
        writer.add_scalar('CELoss/train', train_loss['ce'], epoch)
        writer.add_scalar('MSELoss/train', train_loss['mse'], epoch)
        writer.add_scalar('NCELoss/train', train_loss['nce'], epoch)
        
        writer.add_scalar('CELoss/Valid', val_loss['ce'], epoch)
        writer.add_scalar('MSELoss/Valid', val_loss['mse'], epoch)
        writer.add_scalar('NCELoss/Valid', val_loss['nce'], epoch)
        
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/Valid', val_acc, epoch)
    
        writer.add_scalar('LR', Optimizers['feature'].param_groups[0]['lr'], epoch)
        '''
        if val_acc > best_acc :
            best_acc = val_acc
            #save model
            states = {
                'epoch': epoch + 1,
                'feature_net':Networks['feature'].state_dict(),
                'classifier_net':Networks['classifier'].state_dict(),
                'transformation_net':Networks['transformation'].state_dict(),
                'feature_optimizer': Optimizers['feature'].state_dict(),
                'classifier_optimizer': Optimizers['classifier'].state_dict(),
                'transformation_optimizer': Optimizers['transformation'].state_dict(),
                'best_accuracy': best_acc
            }
            torch.save(states, save_path)
            print('Model Saved')

In [8]:
run_main_loop(loaders, num_epochs)

Performing 0th epoch
normalization constant Z is set to 175837.7
Train Epoch: 0 [0/80000 (0%)]	Loss: CE 1.396792, MSE 0.069122, NCE 9.980989
Train Epoch: 0 [9600/80000 (12%)]	Loss: CE 1.365031, MSE 0.000337, NCE 20.038273
Train Epoch: 0 [19200/80000 (24%)]	Loss: CE 1.368733, MSE 0.000323, NCE 20.951624
Train Epoch: 0 [28800/80000 (36%)]	Loss: CE 1.291469, MSE 0.000053, NCE 23.860016
Train Epoch: 0 [38400/80000 (48%)]	Loss: CE 1.325065, MSE 0.000380, NCE 22.596191
Train Epoch: 0 [48000/80000 (60%)]	Loss: CE 1.314853, MSE 0.000179, NCE 28.642368
Train Epoch: 0 [57600/80000 (72%)]	Loss: CE 1.292187, MSE 0.000150, NCE 29.688772
Train Epoch: 0 [67200/80000 (84%)]	Loss: CE 1.301137, MSE 0.000075, NCE 26.806679
Train Epoch: 0 [76800/80000 (96%)]	Loss: CE 1.272196, MSE 0.000113, NCE 27.883701
Time for epoch pass 245.66650128364563
Train set: Average loss: CE 1.3250, MSE 0.0016, NCE 24.9360, Accuracy: 120272/320000 (38%)

normalization constant Z is set to 22520.2
Valid Epoch: 0 [0/10000 (0%)]	

Time for epoch pass 15.062572717666626
Valid set: Average loss: CE 1.2148, MSE 0.0001, NCE 10.1207, Accuracy: 18180/40000 (45%)

Model Saved
Performing 7th epoch
Train Epoch: 7 [0/80000 (0%)]	Loss: CE 1.175849, MSE 0.000054, NCE 8.169191
Train Epoch: 7 [9600/80000 (12%)]	Loss: CE 1.224514, MSE 0.000084, NCE 8.012356
Train Epoch: 7 [19200/80000 (24%)]	Loss: CE 1.219864, MSE 0.000060, NCE 8.591083
Train Epoch: 7 [28800/80000 (36%)]	Loss: CE 1.263568, MSE 0.000072, NCE 8.475759
Train Epoch: 7 [38400/80000 (48%)]	Loss: CE 1.266302, MSE 0.000083, NCE 8.403940
Train Epoch: 7 [48000/80000 (60%)]	Loss: CE 1.145781, MSE 0.000079, NCE 8.200518
Train Epoch: 7 [57600/80000 (72%)]	Loss: CE 1.206253, MSE 0.000078, NCE 8.582422
Train Epoch: 7 [67200/80000 (84%)]	Loss: CE 1.129377, MSE 0.000085, NCE 8.481626
Train Epoch: 7 [76800/80000 (96%)]	Loss: CE 1.198701, MSE 0.000073, NCE 8.416103
Time for epoch pass 245.7385036945343
Train set: Average loss: CE 1.2047, MSE 0.0001, NCE 8.4611, Accuracy: 146494/

Time for epoch pass 15.293487787246704
Valid set: Average loss: CE 1.1477, MSE 0.0002, NCE 10.1167, Accuracy: 19584/40000 (49%)

Model Saved
Performing 14th epoch
Train Epoch: 14 [0/80000 (0%)]	Loss: CE 1.080898, MSE 0.000201, NCE 3.642040
Train Epoch: 14 [9600/80000 (12%)]	Loss: CE 1.091471, MSE 0.000197, NCE 3.585987
Train Epoch: 14 [19200/80000 (24%)]	Loss: CE 1.177875, MSE 0.000208, NCE 3.661049
Train Epoch: 14 [28800/80000 (36%)]	Loss: CE 1.124004, MSE 0.000208, NCE 3.576352
Train Epoch: 14 [38400/80000 (48%)]	Loss: CE 1.104855, MSE 0.000210, NCE 3.693525
Train Epoch: 14 [48000/80000 (60%)]	Loss: CE 1.150280, MSE 0.000201, NCE 3.645180
Train Epoch: 14 [57600/80000 (72%)]	Loss: CE 1.091142, MSE 0.000232, NCE 3.658311
Train Epoch: 14 [67200/80000 (84%)]	Loss: CE 1.165384, MSE 0.000236, NCE 3.689753
Train Epoch: 14 [76800/80000 (96%)]	Loss: CE 1.215134, MSE 0.000236, NCE 3.739762
Time for epoch pass 250.38363671302795
Train set: Average loss: CE 1.1325, MSE 0.0002, NCE 3.6185, Accura

Valid Epoch: 20 [9600/10000 (94%)]	Loss: CE 0.931369, MSE 0.000430, NCE 10.164165
Time for epoch pass 14.368187189102173
Valid set: Average loss: CE 1.0897, MSE 0.0004, NCE 10.1116, Accuracy: 21138/40000 (53%)

Model Saved
Performing 21th epoch
Train Epoch: 21 [0/80000 (0%)]	Loss: CE 1.039887, MSE 0.000455, NCE 2.365644
Train Epoch: 21 [9600/80000 (12%)]	Loss: CE 1.001643, MSE 0.000482, NCE 2.325134
Train Epoch: 21 [19200/80000 (24%)]	Loss: CE 1.029485, MSE 0.000477, NCE 2.295840
Train Epoch: 21 [28800/80000 (36%)]	Loss: CE 1.085465, MSE 0.000500, NCE 2.238340
Train Epoch: 21 [38400/80000 (48%)]	Loss: CE 1.058864, MSE 0.000501, NCE 2.298044
Train Epoch: 21 [48000/80000 (60%)]	Loss: CE 1.073265, MSE 0.000479, NCE 2.250693
Train Epoch: 21 [57600/80000 (72%)]	Loss: CE 1.122748, MSE 0.000497, NCE 2.334903
Train Epoch: 21 [67200/80000 (84%)]	Loss: CE 1.185729, MSE 0.000473, NCE 2.305613
Train Epoch: 21 [76800/80000 (96%)]	Loss: CE 1.037814, MSE 0.000453, NCE 2.406924
Time for epoch pass 245

Valid Epoch: 27 [0/10000 (0%)]	Loss: CE 1.033857, MSE 0.000528, NCE 10.070802
Valid Epoch: 27 [9600/10000 (94%)]	Loss: CE 0.866496, MSE 0.000625, NCE 10.047035
Time for epoch pass 15.302263498306274
Valid set: Average loss: CE 0.9919, MSE 0.0006, NCE 10.0998, Accuracy: 23342/40000 (58%)

Model Saved
Performing 28th epoch
Train Epoch: 28 [0/80000 (0%)]	Loss: CE 0.955170, MSE 0.000627, NCE 1.817118
Train Epoch: 28 [9600/80000 (12%)]	Loss: CE 1.005415, MSE 0.000664, NCE 1.849491
Train Epoch: 28 [19200/80000 (24%)]	Loss: CE 0.920736, MSE 0.000623, NCE 1.817812
Train Epoch: 28 [28800/80000 (36%)]	Loss: CE 0.990568, MSE 0.000722, NCE 1.817427
Train Epoch: 28 [38400/80000 (48%)]	Loss: CE 0.871156, MSE 0.000660, NCE 1.902157
Train Epoch: 28 [48000/80000 (60%)]	Loss: CE 0.844605, MSE 0.000674, NCE 1.827951
Train Epoch: 28 [57600/80000 (72%)]	Loss: CE 0.882070, MSE 0.000655, NCE 1.866411
Train Epoch: 28 [67200/80000 (84%)]	Loss: CE 0.852774, MSE 0.000691, NCE 1.785032
Train Epoch: 28 [76800/8000

...

Valid Epoch: 181 [0/10000 (0%)]	Loss: CE 1.463034, MSE 0.000396, NCE 10.186736
Valid Epoch: 181 [9600/10000 (94%)]	Loss: CE 0.943949, MSE 0.000401, NCE 10.065641
Time for epoch pass 14.560480833053589
Valid set: Average loss: CE 1.2604, MSE 0.0004, NCE 10.1352, Accuracy: 27982/40000 (70%)

Performing 182th epoch
Train Epoch: 182 [0/80000 (0%)]	Loss: CE 0.006973, MSE 0.000411, NCE 1.246002
Train Epoch: 182 [9600/80000 (12%)]	Loss: CE 0.011680, MSE 0.000406, NCE 1.148368
Train Epoch: 182 [19200/80000 (24%)]	Loss: CE 0.009772, MSE 0.000415, NCE 1.197048
Train Epoch: 182 [28800/80000 (36%)]	Loss: CE 0.008474, MSE 0.000410, NCE 1.250578
Train Epoch: 182 [38400/80000 (48%)]	Loss: CE 0.007294, MSE 0.000415, NCE 1.336917
Train Epoch: 182 [48000/80000 (60%)]	Loss: CE 0.007472, MSE 0.000418, NCE 1.272080
Train Epoch: 182 [57600/80000 (72%)]	Loss: CE 0.007166, MSE 0.000406, NCE 1.306661
Train Epoch: 182 [67200/80000 (84%)]	Loss: CE 0.006707, MSE 0.000417, NCE 1.216155
Train Epoch: 182 [76800/8000

Valid Epoch: 188 [0/10000 (0%)]	Loss: CE 1.463803, MSE 0.000396, NCE 10.184673
Valid Epoch: 188 [9600/10000 (94%)]	Loss: CE 0.949341, MSE 0.000401, NCE 10.073020
Time for epoch pass 14.730249643325806
Valid set: Average loss: CE 1.2591, MSE 0.0004, NCE 10.1361, Accuracy: 27981/40000 (70%)

Performing 189th epoch
Train Epoch: 189 [0/80000 (0%)]	Loss: CE 0.013259, MSE 0.000416, NCE 1.301822
Train Epoch: 189 [9600/80000 (12%)]	Loss: CE 0.006918, MSE 0.000403, NCE 1.181111
Train Epoch: 189 [19200/80000 (24%)]	Loss: CE 0.022340, MSE 0.000426, NCE 1.166087
Train Epoch: 189 [28800/80000 (36%)]	Loss: CE 0.009074, MSE 0.000422, NCE 1.298917
Train Epoch: 189 [38400/80000 (48%)]	Loss: CE 0.011387, MSE 0.000416, NCE 1.285267
Train Epoch: 189 [48000/80000 (60%)]	Loss: CE 0.010602, MSE 0.000414, NCE 1.331371
Train Epoch: 189 [57600/80000 (72%)]	Loss: CE 0.009472, MSE 0.000422, NCE 1.371320
Train Epoch: 189 [67200/80000 (84%)]	Loss: CE 0.009907, MSE 0.000398, NCE 1.281998
Train Epoch: 189 [76800/8000

Valid Epoch: 195 [0/10000 (0%)]	Loss: CE 1.444892, MSE 0.000400, NCE 10.152115
Valid Epoch: 195 [9600/10000 (94%)]	Loss: CE 0.948199, MSE 0.000405, NCE 10.057859
Time for epoch pass 14.843295097351074
Valid set: Average loss: CE 1.2609, MSE 0.0004, NCE 10.1390, Accuracy: 27959/40000 (70%)

Performing 196th epoch
Train Epoch: 196 [0/80000 (0%)]	Loss: CE 0.010510, MSE 0.000409, NCE 1.376850
Train Epoch: 196 [9600/80000 (12%)]	Loss: CE 0.014342, MSE 0.000416, NCE 1.262560
Train Epoch: 196 [19200/80000 (24%)]	Loss: CE 0.014673, MSE 0.000406, NCE 1.251582
Train Epoch: 196 [28800/80000 (36%)]	Loss: CE 0.011323, MSE 0.000420, NCE 1.292669
Train Epoch: 196 [38400/80000 (48%)]	Loss: CE 0.007660, MSE 0.000407, NCE 1.320715
Train Epoch: 196 [48000/80000 (60%)]	Loss: CE 0.010051, MSE 0.000420, NCE 1.376743
Train Epoch: 196 [57600/80000 (72%)]	Loss: CE 0.015702, MSE 0.000421, NCE 1.349696
Train Epoch: 196 [67200/80000 (84%)]	Loss: CE 0.009112, MSE 0.000416, NCE 1.287534
Train Epoch: 196 [76800/8000

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send

KeyboardInterrupt: 

In [9]:
#save model again with different name
save_path = "weights/AlexNet_Decoupling_200.pth"
states = {
                'epoch': 200,
                'feature_net':Networks['feature'].state_dict(),
                'classifier_net':Networks['classifier'].state_dict(),
                'transformation_net':Networks['transformation'].state_dict(),
                'feature_optimizer': Optimizers['feature'].state_dict(),
                'classifier_optimizer': Optimizers['classifier'].state_dict(),
                'transformation_optimizer': Optimizers['transformation'].state_dict()
            }
torch.save(states, save_path)